In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from fastai.vision import *
from fastai import * 
from fastai.text import *
from pathlib import Path 
from tqdm import tqdm
tqdm.pandas(desc="my bar!")

In [3]:
import torchtext
from torchtext.data import Iterator, BucketIterator
from torchvision import transforms, utils
from sklearn.metrics import confusion_matrix, f1_score, auc, roc_curve, precision_recall_curve

In [4]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

# Language Model

In [5]:
lm = True
clean = False

In [6]:
if lm: path = Path('../input'); 
list(path.iterdir())

[WindowsPath('../input/embeddings'),
 WindowsPath('../input/glove.840B.300d'),
 WindowsPath('../input/sample_submission.csv'),
 WindowsPath('../input/test.csv'),
 WindowsPath('../input/test_Clean.csv'),
 WindowsPath('../input/tmp_lm'),
 WindowsPath('../input/train.csv'),
 WindowsPath('../input/train_Clean.csv')]

train_df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')
allText_df = train_df.copy().append(test_df.copy(), sort=False)

In [7]:
if clean:
    train_df = pd.read_csv(path/'train.csv')
    test_df = pd.read_csv(path/'test.csv')
    
    train_df["question_text"] = train_df["question_text"].apply(lambda x: x.lower())
    test_df["question_text"] = test_df["question_text"].apply(lambda x: x.lower())
    #allText_df["question_text"] = allText_df["question_text"].apply(lambda x: x.lower())

    train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_text(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x: clean_text(x))
    #allText_df["question_text"] = allText_df["question_text"].apply(lambda x: clean_text(x))

    train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_numbers(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x:clean_numbers(x))
    #allText_df["question_text"] = allText_df["question_text"].apply(lambda x: clean_numbers(x))

    train_df['question_text'] = train_df['question_text'].apply(lambda x: replace_typical_misspell(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x: replace_typical_misspell(x))
    #allText_df["question_text"] = allText_df["question_text"].apply(lambda x: replace_typical_misspell(x))

    train_x = train_df['question_text'].fillna('_##_').values
    test_x = test_df['question_text'].fillna('_##_').values
    
    train_Full = pd.read_csv(path/'train.csv')
    test_Full = pd.read_csv(path/'test.csv')
    
    train_Full['question_text'] = train_x
    test_Full['question_text'] = test_x
    
    train_Full.to_csv('train_Clean.csv',index=False)
    test_Full.to_csv('test_Clean.csv',index=False)

import zipfile
zf = zipfile.ZipFile(path/'test.csv.zip')
df_test = pd.read_csv(zf.open('test.csv'))
zf = zipfile.ZipFile(path/'train.csv.zip')
df_train = pd.read_csv(zf.open('train.csv'))



In [8]:
#print(df_train.head(3))

print(train_df.shape)
print(test_df.shape)

In [9]:
# df_train.head(3)

In [10]:
def prepare_sequences():
    tokenizer = lambda text: text.split() # the function above is the function we will be using to tokenize the text
    TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer, lower=True)
    LABEL = torchtext.data.Field(sequential=False, use_vocab=False) # sequential and use_vocab=False since no text (binary)
    train_datafields = [("qid", None), ("question_text", TEXT), ("target", LABEL)]
    train = torchtext.data.TabularDataset( # If we had a validation set as well, we would add an additional .splits(...)
                        # path="data/train_cleaned_v2.csv", # the root directory where the data lies
                        path = path/'train_Clean.csv',
                        format='csv',
                        # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                        skip_header=True, 
                        fields=train_datafields)
    test_datafields = [("qid", None),
                     ("question_text", TEXT)] 
    test = torchtext.data.TabularDataset( 
                path=path/'test_Clean.csv',
                format="csv",
                skip_header=True,
                fields=test_datafields)
    return TEXT, LABEL, train, test
# Chat Conversation End
# Type a message...



In [11]:
TEXT,LABEL,train,test = prepare_sequences()


LABEL.batch_first


In [12]:
TEXT.build_vocab(train, test, vectors = "glove.6B.50d")

In [13]:
train_storage = train

In [14]:
#train = train_storage

In [15]:
train,vali = train.split(0.8)

In [16]:
from torchtext.data import Iterator, BucketIterator
train_iter = BucketIterator(
    train, # we pass in the datasets we want the iterator to draw data from
    batch_size= 128, 
    sort_key= lambda x: len(x.question_text), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False, # sorting would add bias
    repeat=False)

vali_iter = BucketIterator(
    vali, # we pass in the datasets we want the iterator to draw data from
    batch_size= 128, 
    sort_key= lambda x: len(x.question_text), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False, # sorting would add bias
    repeat=False)

test_iter = Iterator(
    test,
    batch_size = 128,
    train=False,
    sort = False,
    sort_within_batch = False,
    repeat = False)

ngpu = 0
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

iter(test_iter).__next__().question_text

#len(train_iter)
#label_Size = next(iter(train_iter)).target
#len(label_Size[label_Size==1])
#train_iter.batches
#train_iter = iter(train_iter)
#train_iter.__next__().question_text
#train_iter.__next__().target
#iter(train_iter).__next__().question_text.shape

#smaller,_ = train.split(0.1)
TEXT.vocab.itos[10]

test_iter.dataset.examples.__getitem__(1).__dict__

#smaller.examples[0].question_text
test.examples.__getitem__(2).__dict__

In [17]:
#len(train)
#len(test_iter)

In [18]:
#train.examples[0].question_text

In [19]:
TEXT.vocab.vectors.shape

torch.Size([214945, 50])

TEXT.vocab.freqs

In [20]:
embed_size = 50
hidden_size = 300
ver1 = False
sigBCE = True
bidir = True
layer_LSTM = 2
layer_Lin = 3
dropout_LSTM = 0.1
dropout_Lin = 0.1

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
        
        hidden_size = 64
        
        self.embedding = nn.Embedding(len(TEXT.vocab),embed_size)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors)
        self.embedding_dropout = nn.Dropout2d(0.1)
        
        self.lstm = nn.LSTM(embed_size, hidden_size, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(hidden_size*2, hidden_size, bidirectional=True, batch_first=True)
        
        self.linear = nn.Linear(hidden_size*4,2)
        self.relu = nn.ReLU()
        #self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(16,1)
        
    def forward(self,train):
        #print(train.shape)
        h_embedding = self.embedding(train)
        #print(h_embedding.shape)
        h_embedding = torch.squeeze(self.embedding_dropout(torch.unsqueeze(h_embedding,0)))
        #print(h_embedding.shape)
        
        h_lstm, _ = self.lstm(h_embedding)
        #print(h_lstm.shape)
        h_gru, _ = self.gru(h_lstm)
        #print(h_gru.shape)
        
        avg_pool = torch.mean(h_gru,1)
        #print(avg_pool.shape)
        #avg_pool = avg_pool.view(-1,64)
        max_pool, _ = torch.max(h_gru,1)
        #print(max_pool.shape)
        
        
        conc = torch.cat((avg_pool, max_pool),1)
        print(conc.shape)
        conc = self.relu(self.linear(conc))
        #print(conc.shape)
        #conc = self.dropout(conc)
        print(conc.shape)
        #out = nn.Sigmoid()(conc)
        
        out = nn.LogSoftmax(dim=1)(conc)
        
        #out = self.out(conc)
        #print(out[:,-1].shape)
        
        return out
    
    
    
    
batch = next(iter(train_iter)).question_text
net = NeuralNet()
#net(batch)

In [21]:
#batch = next(iter(train_iter)).question_text
#batch[30]
#layer_Lin

In [22]:
class LSTMTagger(nn.Module):
    def __init__(self):
        super(LSTMTagger, self).__init__()
                      
        self.hidden_dim = hidden_size
        
        self.embedding = nn.Embedding(len(TEXT.vocab),embed_size)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors)
        
        
        #self.embedding_dropout = nn.Dropout2d(0.1)
        
        self.lstm_Layer = nn.LSTM(embed_size, hidden_size,num_layers = layer_LSTM, dropout = dropout_LSTM, bidirectional = bidir)
        
        self.lin_Layer = []
        
        self.lin_Layer_Dropout = dropout_Lin
        
        self.relu = nn.ReLU()
        
        if bidir:
            scale = 2
        else:
            scale = 1
        
        for _ in range(layer_Lin - 1):
            self.lin_Layer.append(nn.Linear(hidden_size*scale,hidden_size*scale))
            self.lin_Layer = nn.ModuleList(self.lin_Layer)
        if sigBCE:
            self.hidden2tag = nn.Linear(hidden_size*scale, 1)
        else:
            self.hidden2tag = nn.Linear(hidden_size*scale, 2)
    
    def forward(self, train):
        #print(train.shape)
        embeds = self.embedding(train)
        #print(embeds.shape)
        #print(embeds.view(len(train),64,-1).shape)
        #print(embeds.view(len(train),train.shape[1],-1).shape)
        lstm_out, _ = self.lstm_Layer(embeds)#embeds.view(len(train),train.shape[1],-1))
        #print(lstm_out.shape)
        
        if ver1:
            #print(lstm_out.shape)
            feature = lstm_out[-1,:,:] # -1 grabs last layer in list
            #print(feature.shape)
        
            for layer in self.lin_Layer:
                feature = layer(feature)
                feature = F.relu(feature)
                predict = self.hidden2tag(feature)
        
            feature = F.relu(feature)
            predict = self.hidden2tag(feature)
            if sigBCE:
                return nn.Sigmoid()(predict)
            else:
                return nn.LogSoftmax(dim=1)(predict)
        else:
           
            feature = torch.mean(lstm_out,0) #avg_pool
            #print(avg_pool.shape)
            for layer in self.lin_Layer:
                feature = layer(feature)
                feature = F.relu(feature)
                
            target_space = self.hidden2tag(feature)#.view(len(train),-1))
            #target_space = self.relu(target_space)
            
            #print(target_space.shape)
            if sigBCE:
                return nn.Sigmoid()(target_space)
            else:
                return F.log_softmax(target_space, dim=1)
            #print(target_score.shape)

inputs_check = next(iter(train_iter)).question_text

#print(inputs_check.shape)

model = LSTMTagger()

with torch.no_grad():
    inputs = inputs_check
    #print(len(inputs))
    score = model(inputs)
    print(score.shape)
    print(score)

torch.Size([128, 1])
tensor([[0.4897],
        [0.4895],
        [0.4895],
        [0.4896],
        [0.4896],
        [0.4895],
        [0.4895],
        [0.4897],
        [0.4895],
        [0.4895],
        [0.4895],
        [0.4896],
        [0.4896],
        [0.4895],
        [0.4896],
        [0.4896],
        [0.4896],
        [0.4895],
        [0.4895],
        [0.4895],
        [0.4896],
        [0.4894],
        [0.4895],
        [0.4895],
        [0.4896],
        [0.4896],
        [0.4895],
        [0.4897],
        [0.4897],
        [0.4895],
        [0.4896],
        [0.4895],
        [0.4895],
        [0.4895],
        [0.4897],
        [0.4896],
        [0.4895],
        [0.4894],
        [0.4895],
        [0.4898],
        [0.4895],
        [0.4897],
        [0.4895],
        [0.4895],
        [0.4895],
        [0.4895],
        [0.4897],
        [0.4895],
        [0.4895],
        [0.4896],
        [0.4895],
        [0.4896],
        [0.4894],
        [0.4894],
       

In [23]:
def weight_init(m):
    if  isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)
        m.bias.data.zero_()

In [24]:
model = LSTMTagger()
model.apply(weight_init)
if sigBCE:
    loss_function = nn.BCELoss(reduction='none')
else:
    loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr=0.2, momentum=0.9)

In [ ]:
validate = True

checking = next(iter(train_iter))
inputs_check = checking.question_text
target_check = checking.target


f1_train = []
auc_train = []
train_accu = []

avg_loss = 0

with torch.no_grad():
    inputs = inputs_check
    score1 = model(inputs)
    #print(score)
    
for epoch in range(2):
    model.train()
    avg_loss = 0
    training_set = iter(train_iter)
    vali_set = iter(vali_iter)
    model.zero_grad()
    all_preds = []
    all_actual = []
    for batch_num in range(len(train_iter)):
        model.train()
        sentence_data = next(training_set)
        vali_data = next(vali_set)
        sentence_in = sentence_data.question_text
        vali_in = vali_data.question_text
        
        
        #target -> target.unsqueeze(1)
        if sigBCE:
            targets = sentence_data.target.float()#.unsqueeze(1)
            vali_target = vali_data.target.float()
        else:
            targets = torch.LongTensor(sentence_data.target)#.unsqueeze(1)
            vali_target = torch.LongTensor(vali.target)
        optimizer.zero_grad()
        
        target_scores = model(sentence_in)
        
        loss = loss_function(target_scores.squeeze(1), targets)#.squeeze(1))#.reshape(64,1).squeeze(1))
        loss = (loss * (targets*17+1)/(targets*17+1).sum()).sum()#.mean()
        loss.backward()
        optimizer.step()
        
        
        model.eval()
        vali_scores = model(vali_in)
        vali_loss = loss_function(vali_scores.squeeze(1), vali_target).mean()
        vali_f1 = (np.asarray(vali_scores.squeeze(1)) > 0.5).astype(int)
        if sigBCE:
            #prediction = np.asarray(target_scores.squeeze(1)>0.5,dtype=float)
            #accuracy = np.asarray(prediction == np.asarray(targets)).sum()/len(targets)*100
            prediction = np.asarray(vali_scores.squeeze(1)>0.5,dtype=float)
            accuracy = np.asarray(prediction == np.asarray(vali_target)).sum()/len(vali_target)*100
        else:
            #prediction = target_scores.data.max(1)[1]
            #accuracy = np.asarray(prediction==targets).sum()/len(targets)*100
            prediction = vali_scores.data.max(1)[1]
            accuracy = np.asarray(prediction == vali_target).sum()/len(vali_target)*100
        #train_accu.append(accuracy)
        
        if sigBCE:
            all_preds.extend(target_scores.tolist())#torch.round(target_scores).tolist())
        else:
            all_preds.extend(torch.exp(target_scores).tolist())#torch.round(target_scores).tolist())
        all_actual.extend(targets.tolist())
        
        #print(np.asarray(target_scores)[:,1].max())
        
        #targets -> targets.reshape(64,1).squeeze(1)
        
        avg_loss += loss.item()
        
        #print(loss)
        if sigBCE:
            print(batch_num+1," of: ",len(train_iter), " Loss: ",loss.item()," F1 Val: ",f1_score(vali_f1, vali_target)," Val Acc: ",accuracy, " High: ", np.asarray(target_scores).max()," Low: ", np.asarray(target_scores).min())
        else:
            print(batch_num+1," of: ",len(train_iter), " Loss: ",loss.item()," Val Loss: ",vali_loss.item()," Acc: ",accuracy, " High: ", np.asarray(torch.exp(target_scores))[:,1].max()," Low: ", np.asarray(target_scores).min())
        
        
    
    #model.eval()
    avg_loss = avg_loss/len(train_iter)
    
    #if validate:
        #avg_val
    
    
    #else:
    print("Training loss for epoch {} is {}".format(epoch + 1, avg_loss))
    
    
    #train_confusion_matrix = confusion_matrix(all_actual, all_preds).ravel()
    if sigBCE:
        pred_Clas = (np.asarray(all_preds) > 0.5).astype(int)#-2.65).astype(int)#
    else:
        pred_Clas = (np.asarray(all_preds)[:,1] > np.asarray(all_preds)[:,0]).astype(int)#-2.65).astype(int)#
    train_confusion_matrix = confusion_matrix(all_actual, pred_Clas).ravel()
    
    print("Confusion matrix for epoch {}, tn={}, fp={}, fn={}, tp={}".format(epoch + 1, *train_confusion_matrix))
    f1_train.append(f1_score(all_actual, pred_Clas))
    print("F1 score for epoch {} is {}".format(epoch + 1, f1_train[-1]))
    fpr, tpr, _ = roc_curve(all_actual, pred_Clas, pos_label=1)
    auc_train.append(auc(fpr, tpr))
    print("AUC for epoch {} is {}\n".format(epoch + 1, auc_train[-1]))

with torch.no_grad():
    inputs = inputs_check
    score2 = model(inputs)
    #print(score)
        
        

1  of:  8164  Loss:  0.688176691532135  F1 Val:  0.13138686131386862  Val Acc:  7.03125  High:  0.50770664  Low:  0.5042168
2  of:  8164  Loss:  0.6758392453193665  F1 Val:  0.1037037037037037  Val Acc:  5.46875  High:  0.5402855  Low:  0.5379057
3  of:  8164  Loss:  0.702202558517456  F1 Val:  0.06060606060606061  Val Acc:  3.125  High:  0.58444995  Low:  0.576483
4  of:  8164  Loss:  0.7081770896911621  F1 Val:  0.1037037037037037  Val Acc:  5.46875  High:  0.6109275  Low:  0.59842134
5  of:  8164  Loss:  0.7098520994186401  F1 Val:  0.11764705882352941  Val Acc:  6.25  High:  0.62048143  Low:  0.60295963
6  of:  8164  Loss:  0.7029476761817932  F1 Val:  0.1037037037037037  Val Acc:  5.46875  High:  0.6173885  Low:  0.5930264
7  of:  8164  Loss:  0.7635444402694702  F1 Val:  0.08955223880597014  Val Acc:  4.6875  High:  0.59926856  Low:  0.5718313
8  of:  8164  Loss:  0.6823981404304504  F1 Val:  0.0  Val Acc:  92.96875  High:  0.5443564  Low:  0.5188275


C:\Users\xw5735\PWinFolder\WPy-3661\python-3.6.6.amd64\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


9  of:  8164  Loss:  0.6906312108039856  F1 Val:  0.0  Val Acc:  90.625  High:  0.5116931  Low:  0.48676753
10  of:  8164  Loss:  0.6824336647987366  F1 Val:  0.0  Val Acc:  92.96875  High:  0.48861262  Low:  0.46397918
11  of:  8164  Loss:  0.7017262578010559  F1 Val:  0.0  Val Acc:  93.75  High:  0.466568  Low:  0.44214916
12  of:  8164  Loss:  0.6652824878692627  F1 Val:  0.0  Val Acc:  93.75  High:  0.45100683  Low:  0.43070987
13  of:  8164  Loss:  0.7276193499565125  F1 Val:  0.0  Val Acc:  96.09375  High:  0.4346541  Low:  0.41468245
14  of:  8164  Loss:  0.637390673160553  F1 Val:  0.0  Val Acc:  92.1875  High:  0.44155023  Low:  0.41554093
15  of:  8164  Loss:  0.6029891967773438  F1 Val:  0.0  Val Acc:  90.625  High:  0.42940304  Low:  0.40427542
16  of:  8164  Loss:  0.8174329996109009  F1 Val:  0.0  Val Acc:  90.625  High:  0.41362154  Low:  0.37470463
17  of:  8164  Loss:  0.7081589102745056  F1 Val:  0.0  Val Acc:  94.53125  High:  0.4351874  Low:  0.38467008
18  of:  816

In [ ]:
vali_data

torch.save(model,"C:/Users/xw5735/PWinFolder/WPy-3661/notebooks/KaggleChal/LSTMTaggerFull.pt")

model = torch.load("C:/Users/xw5735/PWinFolder/WPy-3661/notebooks/KaggleChal/LSTMTaggerFull.pt")

In [ ]:
batch_size = 10
nb_classes = 2
target = torch.LongTensor(batch_size).random_(nb_classes)
#print((targets*5+1))
print((loss * (targets*5+1)).mean())

In [ ]:
prediction = np.asarray(target_scores.squeeze(1)>0.5,dtype=float)
target_scores

In [ ]:
#preddd = np.asarray(all_preds)[:,1]
#preddd_clas = (preddd>-0.45).astype(int)
#train_confusion_matrix = confusion_matrix(all_actual, preddd_clas).ravel()
#print("Confusion matrix for epoch {}, tn={}, fp={}, fn={}, tp={}".format(epoch + 1, *train_confusion_matrix))
#pred_Clas = (np.asarray(all_preds)[:,1] > np.asarray(all_preds)[:,0]).astype(int)
#f1_score(all_actual, pred_Clas)
#pred_Clas

In [ ]:
#preddd = (np.asarray(all_preds)[:,1] > np.asarray(all_preds)[:,0]).astype(int)
#len(preddd[preddd>0])

In [ ]:
def threshold_search(y_true, y_proba, plot=False):
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
    thresholds = np.append(thresholds, 1.001) 
    F = 2 / (1/precision + 1/recall)
    best_score = np.max(F)
    best_th = thresholds[np.argmax(F)]
    if plot:
        plt.plot(thresholds, F, '-b')
        plt.plot([best_th], [best_score], '*r')
        plt.show()
    search_result = {'threshold': best_th , 'f1': best_score}
    return search_result

In [ ]:
def threshold_search_2(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.01 for i in range(100)], disable=True):
        score = f1_score(y_true=y_true, y_pred=y_proba > threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'f1': best_score}
    return search_result

In [ ]:
#threshold_search_2(all_actual, preddd)
#np.asarray(all_preds)[:,1].max()

In [ ]:
#preddd = np.asarray(all_preds)[:,1]
#preddd_clas = (preddd>-0.45).astype(int)
#preddd.max()
#len(preddd_clas[preddd_clas == 1])

In [ ]:
#thr = threshold_search_2(all_actual, np.asarray(all_preds));thr

In [ ]:
#thr = threshold_search(all_actual, np.asarray(all_preds), plot=True);thr

In [ ]:
all_final_preds = []
all_final_rounding = []
model.eval()
testing_batches = iter(test_iter)
for batch_num in range(len(test_iter)):
    final_preds = model(next(testing_batches).question_text)
    prediction = np.asarray(final_preds>0.5,dtype=float)
    print(batch_num+1,"out of: ",len(test_iter)," max: ",np.asarray(final_preds).max())
    all_final_preds.extend(final_preds.tolist())
    all_final_rounding.extend(prediction.tolist())
    

#probab = np.asarray(final_preds)[:,1]

#final_preds_clas = (probab>thr['threshold']).astype(int)

#final_preds_clas.shape

In [ ]:
sub = pd.read_csv(path/'sample_submission.csv')
sub['prediction'] = (np.asarray(all_final_preds)>0.5).astype(int)
sub.to_csv("submission_Kag_Full2.csv",index=False)

In [ ]:
np.asarray(all_final_preds).max()

In [ ]:
testing_batches = iter(test_iter)

In [ ]:
for i, weop in enumerate(test_iter):
    print(i," ",weop.question_text)

In [ ]:
enumerate(test_iter)

In [ ]:
itering = next(testing_batches)

In [ ]:
itering.input_fields

In [ ]:
itering.question_text.shape

In [ ]:
TEXT.vocab.itos[3]

In [ ]:
#thr['threshold']

def train_model(model, train_data, epochs, show=False):
    
    #optimizer = torch.optim.Adam(model.parameters())
    optimizer = optim.SGD(model.parameters(), lr=0.0001)
    #loss_function = nn.BCEWithLogitsLoss(reduction='mean')
    loss_function = nn.NLLLoss()
    
    print("The loss function being used is {}".format(loss_function))
    errors = []
    eval_errors = []
    f1_train = []
    auc_train = []
    f1_eval = []
    auc_eval = []
    
    num_training_batches = len(train_data)
    
    for epoch in range(epochs):
        print("Epoch {}".format(epoch + 1))
        print("Training mode")
        model.train()
        train_iter = iter(train_data)
        model.zero_grad()
        train_loss = 0
        all_preds = []
        all_actual = []
        for batch_num in range(num_training_batches):
            print("Batch {}".format(batch_num + 1))
            print("Out of {}".format(num_training_batches))
            batch = next(train_iter)
            text, class_vector = batch.question_text.transpose(0,1),batch.target.unsqueeze(1)
            #print(batch)
            #print(text.shape)
            #optimizer.zero_grad()
            text_pred = model(text)
            #print(text_pred)
            print(class_vector.shape)
            print(torch.max(class_vector,1)[0].shape)
            loss = loss_function(text_pred,class_vector.reshape(64,1).squeeze(1)) #torch.max(class_vector,1)[0])
            print(loss)
            print(text_pred.squeeze(1))
            print(torch.max(class_vector,1)[0])
            loss.backward()
            optimizer.step()
    

    
    

In [ ]:
#train_iters = iter(train_iter)

In [ ]:
#wee = next(train_iters)

In [ ]:
#wee.question_text.shape